# Numba vs. Cython

- [Numba vs. Cython: Take 2 | Pythonic Perambulations](https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/)

## install numba

In [7]:
!pip install numba

You are using pip version 18.0, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## generate random data

In [8]:
import numpy as np
X = np.random.random((1000, 3))

In [9]:
X

array([[0.22548044, 0.53801418, 0.92863162],
       [0.07979443, 0.870694  , 0.96241506],
       [0.65025958, 0.01420609, 0.93848249],
       ...,
       [0.8137684 , 0.63743494, 0.98231675],
       [0.31523107, 0.38802229, 0.37124034],
       [0.40879012, 0.57537738, 0.40982374]])

In [10]:
X[:,None,:]

array([[[0.22548044, 0.53801418, 0.92863162]],

       [[0.07979443, 0.870694  , 0.96241506]],

       [[0.65025958, 0.01420609, 0.93848249]],

       ...,

       [[0.8137684 , 0.63743494, 0.98231675]],

       [[0.31523107, 0.38802229, 0.37124034]],

       [[0.40879012, 0.57537738, 0.40982374]]])

In [11]:
X[:, None, :] - X

array([[[ 0.        ,  0.        ,  0.        ],
        [ 0.14568602, -0.33267982, -0.03378344],
        [-0.42477914,  0.52380809, -0.00985087],
        ...,
        [-0.58828795, -0.09942076, -0.05368513],
        [-0.08975063,  0.14999189,  0.55739128],
        [-0.18330967, -0.03736321,  0.51880788]],

       [[-0.14568602,  0.33267982,  0.03378344],
        [ 0.        ,  0.        ,  0.        ],
        [-0.57046515,  0.85648791,  0.02393257],
        ...,
        [-0.73397397,  0.23325906, -0.01990169],
        [-0.23543664,  0.48267172,  0.59117472],
        [-0.32899569,  0.29531662,  0.55259132]],

       [[ 0.42477914, -0.52380809,  0.00985087],
        [ 0.57046515, -0.85648791, -0.02393257],
        [ 0.        ,  0.        ,  0.        ],
        ...,
        [-0.16350881, -0.62322885, -0.04383426],
        [ 0.33502851, -0.3738162 ,  0.56724215],
        [ 0.24146947, -0.5611713 ,  0.52865875]],

       ...,

       [[ 0.58828795,  0.09942076,  0.05368513],
        [ 0

## Numpy Boardcasting version of pairwise distance

In [12]:
def pairwise_numpy(X):
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))
%timeit pairwise_numpy(X)

23.7 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Pure Python for-loop version of pairwise distance

In [15]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D
%timeit pairwise_python(X)

2.29 s ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Numba version of pairwise distance

In [16]:
from numba import double
from numba.decorators import jit, autojit

pairwise_numba = autojit(pairwise_python)

%timeit pairwise_numba(X)

2.66 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Cython version of pairwise distance

- [Cython in Ipython: ERROR: Cell magic `%%cython` not found - Stack Overflow](https://stackoverflow.com/questions/36514338/cython-in-ipython-error-cell-magic-cython-not-found)

In [20]:
%load_ext Cython

In [21]:
%%cython

import numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = np.empty((M, M), dtype=np.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return np.asarray(D)

In [22]:
%timeit pairwise_cython(X)

2.85 ms ± 7.14 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


100
